# Natural Language Processing for Signal Generation on News Data

In the following code,  we will utilize pretrained embeddings from both GloVe and FastText Skipgram models to preprocess text datasets for LSTM network.

### Load Packages and Initialize the Environment

In [43]:
import sklearn
import datetime
import pydot, graphviz
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import date
from numpy.random import seed
from IPython.display import Image
from sklearn.model_selection import train_test_split, StratifiedKFold

In [44]:
import keras
import tensorflow as tf
from tensorflow import set_random_seed
from tensorflow.python import pywrap_tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [45]:
from keras import layers
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import LSTM, concatenate, Bidirectional
from keras.layers import PReLU, ELU, LeakyReLU, GRU, SimpleRNN
from keras.layers import Input, Dense, Dropout, Activation, Embedding, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.utils.vis_utils import plot_model

In [46]:
import nltk
nltk.download('stopwords')
from nltk import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marketlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
from tradingcore.utils import embedding_matrix

Setting random state to eliminate randomness. Assigning constant variables for later uses.

In [48]:
seed(42)
set_random_seed(42)
MAX_SEQUENCE_LENGTH = 32
EMBEDDING_DIM = 300

print("Keras version:",keras.__version__)
print("Tensorflow version:",tf.__version__)
print("Sklearn version:",sklearn.__version__)

Keras version: 2.2.4
Tensorflow version: 1.13.1
Sklearn version: 0.20.1


### Load Data - Financial News Dataset 

We will be utilizing open source news data from Bloomberg and Reuters between 2006 and 2012.


In [49]:
df = pd.read_csv("../data/news_data/news_data_labelled.csv")
df.head()

,headline,timestamp,url,tldr,Class
0,Exxon Mobil offers plan to end Alaska dispute,2006-10-20 06:15:00,http://www.reuters.com/article/2006/10/20/busi...,In a proposal sent earlier this week to the Al...,2
1,"Hey buddy, can you spare $600 for a Google share?",2006-10-20 04:25:00,http://www.reuters.com/article/2006/10/20/busi...,SAN FRANCISCO/NEW YORK (Reuters) - Wall Stree...,1
2,Ford posts biggest loss in 14 years,2006-10-23 06:42:00,http://www.reuters.com/article/2006/10/23/us-a...,Ford also said it was considering raising new ...,1
3,Shell looks to buy out Canada unit for C$7.7 b...,2006-10-23 04:34:00,http://www.reuters.com/article/2006/10/23/us-e...,"In July, Shell Canada rattled the industry and...",1
4,"U.S. venture investors betting on energy, Web 2.0",2006-10-23 08:36:00,http://www.reuters.com/article/2006/10/23/us-f...,SAN FRANCISCO (Reuters) - U.S. venture capita...,1


In [50]:
print("Starting timestamp: {}".format(df.timestamp.min()))
print("Ending timestamp: {}".format(df.timestamp.max()))

Starting timestamp: 2006-10-20 04:25:00
Ending timestamp: 2012-12-31 23:06:28


**Split the dataset into testing and training datasets for machine learning**
* X and y respectively correspond to data features (i.e. input) and data labels (i.e. output)
  * Training set the data used to "learn" the parameters in our model with a supervised learning method. This usually uses the majority of the original dataset to achieve best effect.
  * Testing set is the data used to evaluate the effectiveness of our model, often used to produce numerical metrics (e.g. accuracy rate)

In [51]:
X = df.tldr
y = df.Class
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify = y,
                                                    test_size=0.10,
                                                    random_state=42)

### Preprocess Data
<img src="../imgs/preprocess_data.png">

**Tokenize training set**

* Tokenizer from Keras creates a vocabulary index from the training set based on word frequency.
  * Tokenize here also did special character removal for us
  * we can also perform stop word removal here
* Every unique word is assigned a unique integer value

In [52]:
word_filter = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer(num_words = None,
                      filters = word_filter,
                      lower = True,
                      split = " ",
                      char_level = False)

tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

**Convert all datasets to numerical values**
* Apply the vocabulary index to X_train and X_test
  * The datasets are converted from texts to sequences of integers based on previously created vocabulary index
  * The sequences are padded with zeros and are limited with MAX_SEQUENCE_LENGTH to have a fixed length
* Convert y_train and y_test to one-hot encoded vectors

In [53]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen = MAX_SEQUENCE_LENGTH,
                        value = 0.0)

X_test = pad_sequences(tokenizer.texts_to_sequences(X_test),
                       maxlen = MAX_SEQUENCE_LENGTH,
                       value = 0.0)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Build Embeding Matrix

In this project, we will use pretrained word embeding which are stored in files. We have to build the matrices from these files before we use them. Vocabulary index created earlier with the tokenizer is used to create the embedding matrices **embedding_matrix** is a function defined in utils.py which helps us build word embeding from files. It has the following parameters:
* path_to_embedding： path to text file of word embeddings
* embedding_dim: dimension of word embeddings
* word_index: dictionary mapping words to indices

The output is a numpy matrix containing the embeddings

In [54]:
glove_embedding_matrix = embedding_matrix("../data/news_data/glove/glove.840B.300d.txt",
                                          EMBEDDING_DIM,
                                          tokenizer.word_index)

fasttext_embedding_matrix = embedding_matrix("../data/news_data/fasttext/wiki-news-300d-1M.vec",
                                             EMBEDDING_DIM,
                                             tokenizer.word_index)

### Neural Networks Model
Recall from the first notebook that Nerual Network can be treated as a Composite function and each layer is one of these ingredient functions. When we write code to build our own Nerual Network using Keras, what we have to do is just find the layer we want to use and compose (or say concatenate) them. For example to represent the following network:
<img src="../imgs/layer_example.png">

**Network for processing sequential information**
This function build the subnet for processing sequential information. In this project, the input of this subnet is our embeded word vectors.

In [18]:
## Subchannel network for encoding sequential information
def subnetwork_channel(input_layer : layers, RNN_architecture : str, units : int, dropout_rate : float) -> layers:
    """
    This function creates a sub network for encoding sequences.
    
    Inputs:
    input_layer - The input keras layer into the subnetwork
    RNN_architecture - Name of the RNN type to use
    units - Number of units in the RNN
    dropout_rate - dropout rate
    
    Outputs:
    batch - Batch Normalized output layer
    
    """
    assert RNN_architecture in ["LSTM", "GRU", "RNN"]
    
    dropout1 = Dropout(rate = dropout_rate)(input_layer)
    
    if RNN_architecture == "LSTM":
        rnn_layer = Bidirectional(LSTM(units = units, return_sequences = False))(dropout1)
    elif RNN_architecture == "GRU":
        rnn_layer = Bidirectional(GRU(units = units, return_sequences = False))(dropout1)
    elif RNN_architecture == "RNN":
        rnn_layer = Bidirectional(SimpleRNN(units = units, return_sequences = False))(dropout1)
    
    dropout2 = Dropout(rate = dropout_rate)(rnn_layer)
    batch = BatchNormalization()(dropout2)
    return batch

### Network for Generating Signal

In [19]:
## Output layer network
def output_channel(input_layer : layers ,activation : str, units : int, dropout_rate : float) -> layers:
    """
    This function creates a sub network for outputing classification probabilities.
    
    Inputs:
    input_layer - The input keras layer into the subnetwork
    activation  - Name of the activation type to use
    units - Number of units in the Dense network
    dropout_rate - dropout rate
    
    Outputs:
    output - Softmax output layer
    
    """
    assert activation in ["ReLU","PReLU", "ELU", "LeakyReLU"]
    
    dense = Dense(units)(input_layer)
    
    if activation == "PReLU":
        act = PReLU()(dense)
    elif activation == "ELU":
        act = ELU()(dense)
    elif activation == "LeakyReLU":
        act = LeakyReLU()(dense)
    elif activation == "ReLU":
        act = Dense(units, activation='relu')(input_layer)
    
        
    dropout = Dropout(rate = dropout_rate)(act)
    batch = BatchNormalization()(dropout)
    output = Dense(3,activation='softmax', name = "Output")(batch)
    
    return output

### Put All Parts Together

In [20]:
## Define full model.
def define_model(RNN_architecture : str = "LSTM", rnn_units : int = 256, dense_units : int = 128,dense_activation : str = "PReLU" ,dropout_rate : float = 0.4) -> Model:
    """
    This function defines and compiles a Multichannel RNN for Sentiment Classification.
    
    Inputs:
    RNN_architecture - Name of the RNN type to use
    rnn_units - Number of units in the RNN
    dense_units - Number of units in the Dense network
    dense_activation  - Name of the activation type to use
    dropout_rate - dropout rate
    
    Outputs:
    model - A Keras model
    
    """
    # Input Layer
    shape = (MAX_SEQUENCE_LENGTH,)
    input1 = Input(shape = shape, name = "Main_input")
    
    # Channel 1 - GLoVe
    embedding1 = Embedding(len(word_index) + 1,
              EMBEDDING_DIM,
              weights=[glove_embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH,
              trainable=False,
              input_shape=X_train.shape[1:], name = "GLoVe_Embedding")(input1)

    net1 = subnetwork_channel(embedding1, RNN_architecture = RNN_architecture, units = rnn_units, dropout_rate = dropout_rate)
    
    # Channel 2 - Fast Text
    embedding2 = Embedding(len(word_index) + 1,
              EMBEDDING_DIM,
              weights=[fasttext_embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH,
              trainable=False,
              input_shape=shape, name = "FastText_Embedding")(input1)

    net2 = subnetwork_channel(embedding2, RNN_architecture = RNN_architecture, units = rnn_units, dropout_rate = dropout_rate)
    
    # Merge
    merged = concatenate([net1,net2], name ="Merge")
    # Output channel
    output = output_channel(merged, activation = dense_activation, units = dense_units, dropout_rate = dropout_rate)
    
    # Compile 
    model = Model(inputs = input1, outputs = output)
    model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.002), metrics = ['categorical_accuracy'])
    
    return model

### Initialize the Model

In [21]:

model = define_model(RNN_architecture = "LSTM",
                     rnn_units= 256,
                     dense_units = 128,
                     dense_activation = "ReLU",
                     dropout_rate = 0.4)
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Main_input (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
GLoVe_Embedding (Embedding)     (None, 32, 300)      116584800   Main_input[0][0]                 
__________________________________________________________________________________________________
FastText_Embedding (Embedding)  (None, 32, 300)      116584800   Main_input[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)  

In [31]:
pic_name = '../imgs/multichannel-bidirectionalLSTM.png'
plot_model(model,show_shapes=True,to_file=pic_name)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

<img src="../imgs/multichannel-bidirectionalLSTM.png">

### Training

In [28]:
## Train
tensorboard = TensorBoard(log_dir='tasks/tensorboard/logs/')
model.fit(X_train, y_train, epochs = 10, batch_size = 1024, callbacks = [tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
123061/123061 [==============================] - 25s 199us/step - loss: 0.7783 - categorical_accuracy: 0.6743
Epoch 2/10
123061/123061 [==============================] - 23s 183us/step - loss: 0.5505 - categorical_accuracy: 0.7604
Epoch 3/10
123061/123061 [==============================] - 23s 183us/step - loss: 0.5237 - categorical_accuracy: 0.7727
Epoch 4/10
123061/123061 [==============================] - 23s 184us/step - loss: 0.5095 - categorical_accuracy: 0.7791
Epoch 5/10
123061/123061 [==============================] - 23s 183us/step - loss: 0.4987 - categorical_accuracy: 0.7836
Epoch 6/10
123061/123061 [==============================] - 23s 183us/step - loss: 0.4910 - categorical_accuracy: 0.7890
Epoch 7/10
123061/123061 [==============================] - 23s 183us/step - loss: 0.4826 - categorical_accuracy: 0.7935
Epoch 8/10
123061/123061 [==============================] - 23s 185us/step - loss: 0.4767 - categorical_a

### Click [here](/tensorboard/) to start TensorBoard.

### testing

In [29]:
## Find the testing accuracy
val_loss, val_catergorical_accuracy = model.evaluate(X_test,y_test)
print("Validation Accuracy: {:.1f}".format(val_catergorical_accuracy * 100))

13674/13674 [==============================] - 17s 1ms/step
Validation Accuracy: 79.0


Our model was able to achieve ~79% accuracy. According to research on sentiment analysis and classification, human raters may only agree with each other about 80% of the time. Due to the nature of sentiment analysis, the outcome a reader arrives at can be very subjective depending on how the reader interprets the words, tone or phrasing of the text. Thus, a model that predicts with 100% accuracy may still disagree with a human 20% of the time. 

### Exercise: Re-tune Neural Network Parameters
Try experimenting with different parameters in the neural network.
In the function 'define_model'
    - 'RNN_architecture' can be one of: "RNN", "GRU", "LSTM".
    - 'rnn_units' are the number of units in the RNN
    - 'dense_units' are the number of units in the dense network
    - 'dense_activation' can be one of: "PReLU", "LeakyReLU", "ELU", "ReLU"
    - 'dropout_rate' rate of dropout throughout the network